In [115]:
pwd

'/home/sh/lab/etritf'

In [116]:
cd /home/sh/lab/etritf

/home/sh/lab/etritf


In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

In [118]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 2020년
- 기준: deepsleepduration 0.2673794848818764
- 데이터: last.csv
- 예측변수: 'lightsleepduration', 'remsleepduration', 'hr_average',
       'hr_min', 'snoringepisodecount','rr_average', 'breathing_disturbances_intensity'

In [119]:
df_2020 = pd.read_csv('./ja_data/last.csv')
df_2020.head()

,Unnamed: 0,userId,date,endInput,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,...,hr_average,hr_min,hr_max,rr_average,rr_min,rr_max,breathing_disturbances_intensity,snoring,snoringepisodecount,sleep_score
0,0,user01,2020-08-30,2020-08-31 8:43,3.0,6.0,2.0,3.0,3.0,4.0,...,69,57,83,16,12,25,18,8880,16,83
1,1,user01,2020-08-31,2020-09-01 10:18,2.0,3.0,2.0,2.0,3.0,4.0,...,73,63,82,16,12,22,20,8280,16,82
2,2,user01,2020-09-02,2020-09-03 11:04,3.0,3.0,2.0,2.0,3.0,4.0,...,72,64,80,16,11,24,20,11400,11,82
3,3,user01,2020-09-03,2020-09-04 12:33,4.0,3.0,2.0,3.0,3.0,2.0,...,72,60,85,16,12,21,14,5280,5,60
4,4,user01,2020-09-04,2020-09-05 10:25,3.0,3.0,4.0,2.0,3.0,4.0,...,71,61,82,16,11,22,15,5460,11,79


In [120]:
df_2020.columns

Index(['Unnamed: 0', 'userId', 'date', 'endInput', 'sleep', 'sleepProblem',
       'dream', 'amCondition', 'amEmotion', 'pmEmotion', 'pmStress',
       'pmFatigue', 'caffeine', 'cAmount(ml)', 'alcohol', 'aAmount(ml)',
       'startDt', 'endDt', 'lastUpdate', 'wakeupduration',
       'lightsleepduration', 'deepsleepduration', 'wakeupcount',
       'durationtosleep', 'remsleepduration', 'durationtowakeup', 'hr_average',
       'hr_min', 'hr_max', 'rr_average', 'rr_min', 'rr_max',
       'breathing_disturbances_intensity', 'snoring', 'snoringepisodecount',
       'sleep_score'],
      dtype='object')

In [121]:
df_2020['total_sleep_time']=df_2020['lightsleepduration']+df_2020['deepsleepduration']+df_2020['remsleepduration']
for x in ['lightsleepduration', 'remsleepduration', 'hr_average','deepsleepduration',
       'hr_min', 'snoringepisodecount','rr_average', 'breathing_disturbances_intensity']:
    df_2020[x]=df_2020[x]/df_2020['total_sleep_time']

X = df_2020[['lightsleepduration', 'remsleepduration', 'hr_average',
       'hr_min', 'snoringepisodecount','rr_average', 'breathing_disturbances_intensity']]
y = df_2020['deepsleepduration']

In [122]:
print(np.median(y))

0.2673794848818764


In [123]:
# 이진 분류를 위해 y값 변환
y_binary = y.apply(lambda x: 1 if x >= 0.2673794848818764 else 0)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.33, random_state=42)


# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred)
rf_recall = recall_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")

#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred)
xgb_recall = recall_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)


Random Forest Accuracy: 0.9401197604790419
Random Forest Precision: 0.948051948051948
Random Forest Recall: 0.9240506329113924
Random Forest F1-score: 0.9358974358974359
---------------------------------------
XGB Accuracy: 0.9640718562874252
XGB Precision: 0.9620253164556962
XGB Recall: 0.9620253164556962
XGB F1-score: 0.9620253164556962


# 2019년
- 기준: movement_index 4.178000000000001
- 'time_in_bed', 'wakeupcount', 'waso', 'fragmentation_index','aal'

In [124]:
data=pd.read_csv('./ja_data/total1819.csv').drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
data=data.fillna(-1)
data19=data[data['device']=='Actigraph']

In [125]:
data19.columns

Index(['userId', 'night_inputDt', 'pmEmotion', 'pmStress', 'ifUnusual',
       'breakfast', 'lunch', 'dinner', 'lateSnack', 'amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount',
       'date', 'sleep', 'inputDt', 'sleepProblem', 'dream', 'amCondition',
       'amEmotion', 'total_CaffAmount', 'date_m', 'startDt', 'endDt', 'device',
       'sleep_score', 'total_sleep_time', 'time_in_bed', 'waso', 'wakeupcount',
       'aal', 'movement_index', 'fragmentation_index', 'sleep_frag_index'],
      dtype='object')

In [126]:
for x in ['time_in_bed', 'wakeupcount', 'waso', 'aal']:
    data19[x]=data19[x]/data19['total_sleep_time']

X = data19[['time_in_bed', 'wakeupcount', 'waso', 'fragmentation_index','aal']]
y = data19['movement_index']

/tmp/ipykernel_1869622/3884045365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data19[x]=data19[x]/data19['total_sleep_time']


In [106]:
print(np.median(y))

4.178000000000001


In [127]:
# 이진 분류를 위해 y값 변환
y_binary = y.apply(lambda x: 1 if x >=4.178000000000001 else 0)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.33, random_state=42)


# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred)
rf_recall = recall_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred)
xgb_recall = recall_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)


Random Forest Accuracy: 0.968421052631579
Random Forest Precision: 1.0
Random Forest Recall: 0.9433962264150944
Random Forest F1-score: 0.970873786407767
---------------------------------------
XGB Accuracy: 0.9578947368421052
XGB Precision: 0.9803921568627451
XGB Recall: 0.9433962264150944
XGB F1-score: 0.9615384615384616


# 2018년
- 기준: sleepProblem 1
- 'time_in_bed','lunch','pmStress','dinner', 'sleep'

In [128]:
data=pd.read_csv('./ja_data/total1819.csv').drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
data=data.fillna(-1)
data18=data[data['device']=='Fitbit']
data18=data18.drop(['waso', 'wakeupcount','aal','movement_index','fragmentation_index', 'sleep_frag_index','userId', 'night_inputDt', 'amCaffeine','pmCaffeine','alcohol','device','inputDt','date','date_m','startDt', 'endDt'],axis=1)

In [129]:
data18.head()

,pmEmotion,pmStress,ifUnusual,breakfast,lunch,dinner,lateSnack,amCaffAmount,pmCaffAmount,aAmount,sleep,sleepProblem,dream,amCondition,amEmotion,total_CaffAmount,sleep_score,total_sleep_time,time_in_bed
0,3.0,1.0,2.0,1.0,3.0,3.0,4.0,-1.0,-1.0,-1.0,3.0,3.0,2.0,4.0,5.0,-1.0,90.800478,22800,25110
1,5.0,4.0,2.0,1.0,3.0,3.0,-1.0,2.0,-1.0,-1.0,3.0,0.0,4.0,3.0,4.0,-1.0,91.984231,21000,22830
2,3.0,2.0,2.0,2.0,3.0,3.0,4.0,-1.0,1.0,-1.0,4.0,6.0,4.0,2.0,2.0,-1.0,88.482385,19590,22140
3,4.0,2.0,2.0,2.0,3.0,3.0,4.0,1.0,1.0,-1.0,3.0,0.0,4.0,1.0,5.0,2.0,88.781665,22080,24870
4,3.0,2.0,2.0,1.0,3.0,3.0,2.0,-1.0,-1.0,-1.0,4.0,0.0,4.0,1.0,2.0,-1.0,92.403101,17880,19350


In [39]:
data18.columns

Index(['pmEmotion', 'pmStress', 'ifUnusual', 'breakfast', 'lunch', 'dinner',
       'lateSnack', 'amCaffAmount', 'pmCaffAmount', 'aAmount', 'sleep',
       'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'total_CaffAmount',
       'sleep_score', 'total_sleep_time', 'time_in_bed'],
      dtype='object')

In [130]:
for x in ['time_in_bed','lunch','pmStress','dinner', 'sleep']:
    data18[x]=data18[x]/data18['total_sleep_time']
X = data18[['time_in_bed','lunch','pmStress','dinner', 'sleep']]
y = data18['sleepProblem']

In [131]:
# 이진 분류를 위해 y값 변환
y_binary = y.apply(lambda x: 1 if x >= 1 else 0)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.33, random_state=42)


# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred)
rf_recall = recall_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")

#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred)
xgb_recall = recall_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)


Random Forest Accuracy: 0.5972222222222222
Random Forest Precision: 0.7222222222222222
Random Forest Recall: 0.5777777777777777
Random Forest F1-score: 0.6419753086419753
---------------------------------------
XGB Accuracy: 0.5416666666666666
XGB Precision: 0.6578947368421053
XGB Recall: 0.5555555555555556
XGB F1-score: 0.6024096385542169
